In [4]:
import os
from autogen import ConversableAgent

In [248]:
# !pip install markdownify

In [7]:
from autogen.browser_utils import SimpleTextBrowser

In [249]:
# Autogen - Not useful in current requirement
browser = SimpleTextBrowser()
browser.address

'about:blank'

# Util Functions

In [280]:
def load_transcript(filename):
    loader = PyPDFLoader(os.path.join(os.getcwd(), "ceat/transcripts", filename))
    pages = loader.load_and_split()
    return pages

def print_pdf(pdf_pages):
    print("x" * 100)
    for i in pdf_pages:
        print(i.page_content)
        print("x" * 100)
    return 

# Response returned is a set of multiple json objects concatenated as a string separated by newline.
# The long string with multiple JSON objects separated by newlines

def load_res_into_dict(res):
    response_string = response.text

    # Split the string into a list of strings, each containing a JSON object
    json_strings = response_string.strip().split('\n')

    # Initialize a list to hold the parsed JSON objects
    json_objects = []

    # Iterate over each string in the list and parse it as JSON
    for json_str in json_strings:
        try:
            json_object = json.loads(json_str)
            json_objects.append(json_object)
        except json.JSONDecodeError as e:
            print(f"Failed to parse JSON: {e}")
            # Handle or log the error here

    # Now you have a list of dictionaries representing the JSON objects
    # print(json_objects)  # This will show you all the parsed JSON objects
    return json_objects


# Selenium

In [92]:
# !pip install selenium

In [80]:
os.listdir('./chromedriver_mac_arm64')

['chromedriver', 'LICENSE.chromedriver']

## Download Reports from Ceat Website

In [91]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
import time

# Set up the WebDriver
download_dir = os.getcwd() + "/ceat"  # Change this to your desired download folder
print(download_dir)
os.makedirs(download_dir, exist_ok=True)

# Set Chrome preferences to automate downloads
options = Options()
prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,  # To automatically download the PDF
        # "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True  # It will not open the PDF in the browser
    }
options.add_experimental_option("prefs", prefs)
# options.add_argument("--headless")

    # Initialize WebDriver with options
    # service = Service(executable_path="/path/to/chromedriver")  # Update the path to where you've downloaded chromedriver


service = Service(executable_path="./chromedriver_mac_arm64/chromedriver")
driver = webdriver.Chrome(options=options)
print(driver)

# URL of the webpage
url = 'https://www.ceat.com/investors/annual-reports.html'

# Navigate to the URL
driver.get(url)

# Extract the specific component
# Replace 'your-css-selector' with the actual CSS selector for your component
elements = driver.find_elements(By.CSS_SELECTOR, 'a.btn-icon')

report_urls = [element.get_attribute('href') for element in elements]
print("report-urls", report_urls)

if report_urls:
    for report_url in report_urls:
        driver.get(report_url)
        WebDriverWait(driver, 10)
    

# download_pdf(driver, report_urls[0])    # driver.get(report_urls[0])
print("postwait")

time.sleep(60)
driver.quit()

/Users/paruljuniwal/learning_development_Interview/product_suite/ai/fin-robo/ceat
<selenium.webdriver.chrome.webdriver.WebDriver (session="1777383c826f161e4017f466c58b944c")>
report-urls ['https://www.ceat.com/content/dam/ceat/pdf/Annual_Reports/CEAT-Limited-Integrated-Annual-Report-FY23.pdf', 'https://www.ceat.com/content/dam/ceat/pdf/CEAT%20AR%202021-22_Spread-27.6.22.pdf', 'https://www.ceat.com/content/dam/ceat/pdf/Annual_Reports/Ceat-Limited-Integrated-Annual-Report-FY21.pdf', 'https://www.ceat.com/content/dam/ceat/pdf/Annual_Reports/CEAT%20Limited%20Integrated%20Annual%20Report%20FY20.pdf', 'https://www.ceat.com/content/dam/ceat/pdf/Annual_Reports/CEAT%20Limited%20Annual%20Report%20FY19.pdf', 'https://www.ceat.com/content/dam/ceat/pdf/Annual_Reports/CEAT%20Limited%20Annual%20Report%20FY18.pdf', 'https://www.ceat.com/content/dam/ceat/pdf/Annual_Reports/CEAT%20Limited%20Annual%20Report%20FY17.pdf', 'https://www.ceat.com/content/dam/ceat/pdf/Annual_Reports/CEAT%20Limited%20Annual%20R

## Upload PDF files and convert to structured data

In [105]:
reports = [report for report in os.listdir("./ceat") if report.endswith(".pdf")]

print("length:", len(reports), "::::All reports: ", reports)

length: 8 ::::All reports:  ['CEAT Limited Annual Report FY17.pdf', 'CEAT Limited Annual Report FY16.pdf', 'CEAT Limited Integrated Annual Report FY20.pdf', 'CEAT Limited Annual Report FY18.pdf', 'CEAT Limited Annual Report FY19.pdf', 'CEAT AR 2021-22_Spread-27.6.22.pdf', 'Ceat-Limited-Integrated-Annual-Report-FY21.pdf', 'CEAT-Limited-Integrated-Annual-Report-FY23.pdf']


In [106]:
from langchain_community.document_loaders import PyPDFLoader

In [109]:
reports[0]

'CEAT Limited Annual Report FY17.pdf'

In [112]:
loader = PyPDFLoader(os.path.join(os.getcwd(), "ceat", reports[0]))
pages = loader.load_and_split()

In [113]:
len(pages)

400

In [123]:
pages[0].page_content

'TOWARDS A SAFER, \nSMARTER TOMORROW\nAnnual Report 2016-17'

In [ ]:
from pprint import pprint
pprint(pages[1].page_content)

In [239]:
# for i in range(0, 30):
#     print("*" * 100)
#     pprint(pages[i].page_content)
#     print("*" * 100)

In [130]:
from autogen import ConversableAgent, AssistantAgent, UserProxyAgent

In [129]:
local_llama2_uncensored = {
    "model": "llama2-uncensored",
    "base_url": "http://localhost:11434/v1/",
    "temperature": 0.9
    # "api_type": "openai"
}
local_llama3 = {
    "model": "llama3",
    "base_url": "http://localhost:11434/v1/",
    "temperature": 0.2,
    "cache_seed": 42,
    # "api_type": "openai"
}

In [229]:
# assistant = AssistantAgent(
#     name="Assistant",
#     system_message="Only use the tools you have been provided with. Reply TERMINATE when the task is done.",
#     llm_config={"config_list": config_list, "cache_seed": None},
# )

# user_proxy = UserProxyAgent(
#     name="user_proxy",
#     is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
#     human_input_mode="TERMINATE",
#     max_consecutive_auto_reply=10,
#     # code_execution_config={"executor": code_executor},
# )

user_proxy = ConversableAgent(
    name="Editor",
    system_message="""You are the admin editor overseeing the analysis. 
    Interact with the financial analyst agent to discuss the analysis. 
    Provide a feedback to the financial analyst's content on the following lines:
    1. A detailed explantion of the points
    2. No NSFW language""",
    code_execution_config=False,
    human_input_mode="ALWAYS"
)


financial_analyst = ConversableAgent(
    name="Equity Analyst",
    system_message="""You’re an Equity analyst with expertise in auto and auto parts. 
    You have analysed annual developement in all tyre companies in India.  
    How do you think work done by CEAT can help to grow its product portfolio, customer delight, growth in overall revenue 
    and profitability?""",
    
    # llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    llm_config={"config_list": [local_llama3]},
    human_input_mode="NEVER",  # Never ask for human input.
)

# Your name is Cathy. You are the most seasoned financial analyst with 
#     lots of expertise in stock market analysis and investment strategies in a company providing portfolio management 
#     and financial consultancy services for High Networth Individuals. 
#     You are tasked to analyse the report provided for a company's financials and write engaging blogpost.
#     You try generate the best blogpost possible for the user's request. 
#     If the user provides critique, respond with a revised version of your previous attempts.""",

In [194]:
def get_page(idx):
    return pages[idx]

In [230]:
# first_query = ""
# for i in pages[10:27]:
#     first_query += i.page_content

# print(first_query)

In [231]:
task = f"""Give me a detailed analysis of the following text from the financial report. 
Wait for me to analyse your analyse and add comments. Revise your analysis as per my comments.

The content is as follows :\n {first_query}
"""

In [196]:
# print(task)

In [241]:
chat = user_proxy.initiate_chat(
        financial_analyst,
        message=task,
        max_turns=3,
        # cache=cache,
    )

In [166]:
print(len(chat.chat_history))

6


In [214]:
# for i in chat.chat_history:
#     print(i)
#     print("*" * 100)

In [215]:
# print(chat.chat_history[1]["content"])

In [216]:
# print(chat.chat_history[0]["content"])

In [ ]:
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke and then say goodbye.", max_turns=3)

In [ ]:
# return Agent(
#       role='The Best Financial Analyst',
#       goal="""Impress all customers with your financial data 
#       and market trends analysis""",
#       backstory="""The most seasoned financial analyst with 
#       lots of expertise in stock market analysis and investment
#       strategies that is working for a super important customer.""",
#       verbose=True,

In [242]:
# print(pages[11].page_content)

In [243]:
# print(pages[12].page_content)

## Factual Equity Analyst

In [237]:
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="""You are the admin editor overseeing the analysis and query answering. 
    Interact with the financial analyst agent to query factual data the analysis. """,
    code_execution_config=False,
    human_input_mode="NEVER"
)


financial_analyst = AssistantAgent(
    name="Equity Analyst",
    system_message= """You’re an Equity analyst with expertise in auto and auto parts in India. 
    You have analysed annual developement in all tyre companies in India. 
    You will be provided with a document delimited by triple backticks ```  and a question. 
    Your task is to answer the question using only the provided document and to cite the passage(s) 
    of the document used to answer the question. If the document does not contain the information needed 
    to answer this question then simply write: "Insufficient information.
    If an answer to the question is provided, it must be annotated with a citation. 
    Use the following format for to cite relevant passages ({"citation": …}). """,
    
    # llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    llm_config={"config_list": [local_llama3]},
    human_input_mode="NEVER",  # Never ask for human input.
)

In [222]:
def create_prompt(page_content="", question_content=""):
    prompt_template = f""" ```{page_content}``` Question: {question_content}"""
    return prompt_template

In [219]:
task1 = f""" ```{pages[12].page_content}``` Question: What is the PAT for the year 2014 and 2015?"""
task2 = f""" ```{pages[12].page_content}``` Question: What is the PAT for the year 2014 and 2015?"""


In [235]:
task2 = create_prompt(pages[12].page_content, "What is the EPS for the year 2017?")
# print(task2)

In [238]:
chat = user_proxy.initiate_chat(
        financial_analyst,
        message=task2,
        max_turns=3,
        # cache=cache,
)

# Simple Chat interface with llama3

## Transcripts of Conference Call - Jan 2024

In [247]:
# Files
transcripts = os.listdir("./ceat/transcripts/")
transcripts

['Emkay-CEAT-25Jan-2024.pdf']

In [251]:
transcript_24 = load_transcript(transcripts[0])

In [260]:
# print_pdf(transcript_24)

### Simple Chat Setup with Llama3 hosted locally

In [261]:
import json

In [289]:



You will be provided with a thesis abstract and a suggested title for it. 
The thesis title should give the reader a good idea of the topic of the thesis but should also be eye-catching. If the title does not meet these criteria, suggest 5 alternatives.

In [293]:
# print(json.loads(response.text))

In [291]:
response_dict = load_res_into_dict(response.text)
print(len(response_dict))

367


In [295]:
type(transcript_24)
# transcript_24[0].page_content

list

In [315]:
transcript_24_str = "".join([f':::{doc.page_content}:::' for doc in transcript_24]) + "Question: What is the total cpex for FY24?"

In [316]:
transcript_24_str[0:2]

'::'

In [326]:
response = requests.post("http://localhost:11434/api/generate", data=json.dumps({
  "model": "llama3",
  "system": """You are a seasoned financial analyst with lots of expertise in stock market analysis, company financials and investment
      strategies that is working for a super important customer. 
      You will be provided with a transcript of earnings call of company named CEAT. 
      Use the provided transcripts delimited by triple colons ::: to answer questions.
      Go step by step and analyse each transcript individually before answering.
      If the answer cannot be found in the articles, write "I could not find an answer.""",
  "prompt": transcript_24_str,
}))

In [327]:
response_dict = load_res_into_dict(response.text)
print(len(response_dict))

99


In [293]:
# print(response_dict)

In [328]:
answer = [element.get("response") for element in response_dict]
print("".join(answer))

The answer is not provided in the transcript. However, we can see that CEAT Limited has planned investments in new product development, consumer outreach in local markets, and capacity expansion to drive growth. The company also mentions a 5-year road map for product planning, but specific details about capital expenditures (capex) are not mentioned. To get an answer to this question, I would recommend contacting CEAT Limited's investor relations department or reviewing their annual report or financial statements for FY24.


## Finetuning

- Reference: https://colab.research.google.com/drive/135ced7oHytdxu3N2DNe1Z0kqjyYIkDXp?usp=sharing

In [ ]:
alpaca_prompt = """
Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
### Instruction:
{}

### Input:
{}

### Response:
{}
"""

## Different prompt templating languages available including ChatML
## Use SFTTrainer - SupervisedFintuning Trainer from HuggingFace Transformers


# Create a Dataset using LLM for finetuning llama3
- """Create a dataset that contains concept prompt pair. For each of the concept like 'Current EBITDA', create a detailed description that can be used by a GPT working as financial analyst to analyse the concept and answer user queries."""

In [2]:
import os

In [9]:
os.listdir()

['.DS_Store',
 'ceat',
 'chromedriver_mac_arm64',
 'README.md',
 'chromedriver_mac_arm64.zip',
 '.gitignore',
 'QuantumAlpha.ipynb',
 '.ipynb_checkpoints',
 '.git',
 '.cache']

# 7th May 2024: RAG pipeline fro CEAT reports - V1

In [1]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma


In [288]:
DATA_PATH = "ceat"
CHROMA_PATH = "chroma"

In [13]:
def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

In [6]:
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

In [7]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [15]:
def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=get_embedding_function()
    )

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)
    
    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        # print("new_chunk_ids:", new_chunk_ids.split(".pdf:")[-1])
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")

In [14]:
def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

In [10]:
from langchain_community.embeddings.ollama import OllamaEmbeddings

In [11]:
def get_embedding_function():
    embeddings = OllamaEmbeddings(model="llama3", base_url="http://localhost:11434")
    return embeddings

In [ ]:
def main(clear_db=false):
    if (clear_db):
        print("✨ Clearing Database")
        clear_database()
    # Create or update data store
    documents = load_documents()
    chunks = split_documents(documents)
    add_to_chroma(chunks)

# Testing individual functions

In [16]:
documents = load_documents()

In [17]:
len(documents)

1772

In [36]:
documents[316]

Document(page_content='CEAT LIMITED\nRPG HOUSE463, DR ANNIE BESANT ROAD, WORLI  MUMBAI - 400 030\nWWW.CEAT .COM\nMaking \nMobility Safer & S\nMarter. \nevery d\nay.\nMaking Mobility Safer & S Marter. every day.', metadata={'source': 'ceat/CEAT Limited Annual Report FY16.pdf', 'page': 0})

In [37]:
chunks = split_documents(documents)

In [38]:
len(chunks)

11455

In [54]:
chunks[13].metadata

{'source': 'ceat/CEAT Limited Annual Report FY17.pdf',
 'page': 6,
 'id': 'ceat/CEAT Limited Annual Report FY17.pdf:6:2'}

In [65]:
add_to_chroma(chunks)

Number of existing documents in DB: 0
👉 Adding new documents: 11455


In [56]:
import requests
import json

In [15]:
url = 'http://localhost:11434/api/generate'

# Data payload as a dictionary
data = {
    "model": "llama3",
    "prompt": "Why is the sky blue?"
}

headers = {'Content-Type': 'application/json'}

# Make a POST request
response = requests.post(url, data=json.dumps(data), headers=headers)

# Print the response text (JSON response)
# print(response.text)

In [58]:
e = get_embedding_function()

In [59]:
r2 = e.embed_query("What is the second letter of Greek alphabet")

In [61]:
print(len(r2))

4096


In [62]:
chunk_embed = e.embed_documents(chunks[0:2])

In [64]:
print(len(chunk_embed[0]))

4096


### Combined Text for Embedding:
"China accounts for one-third of the global tyre industry and developments there have huge global implications. 
The Chinese tyre market faced tough challenges. Tags: China tyre market, global impact, market challenges."

In [352]:
# for i in range(0, 10):
#     print("*" * 100, "page:", i+1)
#     print(documents[i].page_content)
#     print("*" * 100)

In [76]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
# The following is an excerpt from annual financial report of a tyre manufacturing company called CEAT in India.
# Annotate and label the following excerpt in triple quotes in a structured way so that it can be used in RAG.

In [164]:
PROMPT_TEMPLATE = """
You are a equity research analyst.
You have to structure, annotate and tag the provided annual financial report of a company CEAT which manufactures tyres in India. 
The annotated text with tags with be used to create embeddings for Retrieval Augmented Generation.
Analyse the excerpt and provide a list of comprehensive annotations and tags for the given document.

{excerpt}

---
Use the tags provided in the structure as follows:
{tags}

Provide the structured, annotated and labelled response as json
"""


In [165]:
def create_prompt(excerpt, prompt_template):
    excerpt_text = f'"""{excerpt}"""'
    prompt_template = ChatPromptTemplate.from_template(prompt_template)
    prompt = prompt_template.format(excerpt=excerpt_text, tags=tags)
    # print(prompt)
    # print("*" * 100)
    return prompt
    

In [155]:
annotated_json_texts_v2 = []

In [353]:
model = Ollama(model="llama3", base_url="http://localhost:11434")
for doc in documents[0:10]:
    annotation_prompt = create_prompt(doc.page_content, PROMPT_TEMPLATE)
    annotated_text = model.invoke(annotation_prompt)

    # json_prompt = create_prompt(annotated_text, JSON_PARSER_PROMPT_TEMPLATE)
    # json_text = model.invoke(json_prompt)
    # print("json_text:", json_text)
    # annotated_dict = json.loads(json_text)
    annotated_json_texts_v2.append(annotated_text)
    

In [148]:
len(annotated_json_texts)

9

In [354]:
# for i in annotated_json_texts:
#     print(i)
#     print("x" * 100)

In [83]:
from langchain_community.llms import Ollama

In [355]:
# print(response_text.split("```")[1])

In [132]:
# response_text

In [122]:
JSON_PARSER_PROMPT_TEMPLATE = """
Extract and return the json present in the following text present in triple quotes. Do not return any other text like 'here is the returned json'. Only return pure json

{excerpt}
----------
{output}

"""

In [133]:
# json_prompt = create_prompt(response_text, JSON_PARSER_PROMPT_TEMPLATE)

In [134]:
# res = model.invoke(json_prompt)
# print(res)

In [135]:
# json.loads(res)

In [14]:
# for element in annotated_json_texts_v2:
#     print("x" * 100)
#     print(element)
#     print("x" * 100)
    

In [159]:
tags = {
    "ExecutiveSummary": {
        "Annotations": "Brief overview of the year's achievements, financial health, and strategic directions.",
        "Tags": ["Executive Summary", "Annual Highlights", "CEO Message"]
    },
    "FinancialStatements": {
        "Annotations": "Detailed reports including income statements, balance sheets, cash flow statements, and equity changes.",
        "Tags": ["Financial Statements", "Income Statement", "Balance Sheet", "Cash Flow"]
    },
    "FinancialPerformanceAnalysis": {
        "Annotations": "In-depth analysis of financial results, including profitability, revenue growth, expense management, and earnings.",
        "Tags": ["Financial Performance", "Revenue Growth", "Profitability Analysis", "Expense Management"]
    },
    "MarketAndIndustryOverview": {
        "Annotations": "Analysis of the market conditions, industry trends, competitive landscape, and economic impacts.",
        "Tags": ["Market Overview", "Industry Trends", "Competitive Landscape", "Economic Impact"]
    },
    "RiskManagement": {
        "Annotations": "Discussion on financial and operational risks, mitigation strategies, and regulatory compliance.",
        "Tags": ["Risk Management", "Operational Risks", "Regulatory Compliance", "Risk Mitigation"]
    },
    "CorporateGovernance": {
        "Annotations": "Information on corporate governance practices, board composition, and executive management.",
        "Tags": ["Corporate Governance", "Board of Directors", "Governance Practices"]
    },
    "ResearchAndDevelopment": {
        "Annotations": "Insights into R&D activities, innovation efforts, and new product development.",
        "Tags": ["Research and Development", "Innovation", "Product Development"]
    },
    "StrategicInitiatives": {
        "Annotations": "Details on strategic plans, mergers and acquisitions, market expansion, and new business ventures.",
        "Tags": ["Strategic Initiatives", "Mergers and Acquisitions", "Market Expansion"]
    },
    "SustainabilityAndCSR": {
        "Annotations": "Efforts and achievements in sustainability, environmental impact, corporate social responsibility initiatives.",
        "Tags": ["Sustainability", "Environmental Impact", "Corporate Social Responsibility"]
    },
    "HumanResources": {
        "Annotations": "Updates on human resources policies, employee relations, new hires, training and development.",
        "Tags": ["Human Resources", "Employee Relations", "Training and Development"]
    },
    "LegalProceedings": {
        "Annotations": "Information on any significant legal issues or proceedings involving the company.",
        "Tags": ["Legal Proceedings", "Regulatory Issues", "Compliance"]
    },
    "FutureOutlook": {
        "Annotations": "Projections for the future performance, expected market conditions, and strategic directions.",
        "Tags": ["Future Outlook", "Performance Projections", "Market Projections"]
    },
    "AppendicesAndSupplementaryInformation": {
        "Annotations": "Additional data, methodological explanations, glossaries, and other supplementary details.",
        "Tags": ["Appendices", "Supplementary Information", "Glossaries"]
    }
}


In [163]:
tags["ExecutiveSummary"]["Tags"]

['Executive Summary', 'Annual Highlights', 'CEO Message']

In [169]:
annotated_json_texts_v3 = []

model = Ollama(model="llama3", base_url="http://localhost:11434")
for doc in documents[0:100]:
    annotation_prompt = create_prompt(doc.page_content, PROMPT_TEMPLATE)
    annotated_text = model.invoke(annotation_prompt)

    # json_prompt = create_prompt(annotated_text, JSON_PARSER_PROMPT_TEMPLATE)
    # json_text = model.invoke(json_prompt)
    # print("json_text:", json_text)
    # annotated_dict = json.loads(json_text)
    annotated_json_texts_v3.append(annotated_text)
    

In [16]:
# print(annotated_json_texts_v3[0])

In [172]:
file_path = 'annotated_text.json'

# Write JSON data to a file
with open(file_path, 'w') as file:
    json.dump(annotated_json_texts_v3, file, indent=4)

print(f"Data successfully written to {file_path}")


Data successfully written to annotated_text.json


In [356]:
# for i in annotated_json_texts_v3[0:20]:
#     print(extract_json_from_string(i))
#     print("*"*100)

In [176]:
def extract_json_from_string(s): 
    start = s.find('```') 
    end = s.rfind('```') + 3 
    return s[start:end]

In [262]:
# for i in annotated_json_texts_v3[0:20]:
#     print("x" * 100)
#     print(i)
#     print("x" * 100)

In [259]:
example_text = """Chinese tyre market
China is the world’s largest vehicle market. In 2015, China
recorded a rise of 6% in Passenger Vehicle (PV) sales,
while sales of Commercial Vehicles (CV) dropped 9%.
China accounts for one-third of the global tyre industry
and developments there have huge global implications. In
FY 2015-16, the Chinese tyre market faced tough challenges.
A combination of overcapacities, anti-dumping probes
in major consumer markets such as the U.S., and weaker
domestic demand has made Chinese tyre manufacturers
sell in a distressed manner. This desperation is disrupting
the other markets. China has been the biggest threat to
Indian tyre manufacturers, especially in the TBR (TruckBus Radial) segment. In the absence of anti-dumping duties
being in place, China has been able to supply TBR almost
at the cost of TBB (Truck-Bus Bias) tyres. This has
earned it a healthy market share in India’s TBR replacement
market.
(Source: European Automobile Manufacturers Association)"""

reference_output_with_tags = """Section: Chinese Tyre Market
Title: "Chinese Tyre Market"

Content Summary: Overview of the tyre market dynamics in China, highlighting challenges and the impact on global and Indian markets.

Details:
Market Size: China is described as the world’s largest vehicle market, accounting for one-third of the global tyre industry.
Sales Trends: In 2015, there was a 6% rise in Passenger Vehicle sales, while Commercial Vehicle sales dropped by 9%.
Challenges: The Chinese tyre market faced significant challenges in FY 2015-16, including overcapacities, anti-dumping probes by the U.S., and weaker domestic demand.
Impact: These challenges have led Chinese manufacturers to sell tyres in a distressed manner, disrupting other markets, particularly the Indian TBR (Truck-Bus Radial) market.
Competitive Dynamics: Without anti-dumping duties, China has been able to gain a substantial market share in India’s TBR replacement market by pricing TBR tyres competitively against TBB (Truck-Bus Bias) tyres.
Tags: ["China tyre market", "global impact", "anti-dumping", "market disruption", "Indian tyre market competition"]
"""


In [190]:
PROMPT_TEMPLATE_WITH_EXAMPLE = """
You are a equity research analyst.
You have to structure, annotate and tag the provided annual financial report of a company CEAT which manufactures tyres in India. 
The annotated text with tags with be used to create embeddings for Retrieval Augmented Generation.
Analyse the excerpt and provide a list of comprehensive annotations and tags for the given document.

{excerpt}

---
Use the example input text and format of reference output with tags provided below:
{example_text}
{reference_output}

Provide the structured, annotated and labelled response as json
"""


In [191]:
def create_prompt(excerpt, example_text, reference_output, prompt_template):
    excerpt_text = f'"""{excerpt}"""'
    prompt_template = ChatPromptTemplate.from_template(prompt_template)
    prompt = prompt_template.format(excerpt=excerpt_text, example_text=example_text, reference_output=reference_output)
    print(prompt)
    print("*" * 100)
    return prompt
    

In [198]:
annotated_text_with_reference = []

In [17]:
model = Ollama(model="llama3", base_url="http://localhost:11434")
for doc in documents:
    annotation_prompt = create_prompt(doc.page_content, example_text, reference_output_with_tags, PROMPT_TEMPLATE_WITH_EXAMPLE)
    annotated_text = model.invoke(annotation_prompt)

    # json_prompt = create_prompt(annotated_text, JSON_PARSER_PROMPT_TEMPLATE)
    # json_text = model.invoke(json_prompt)
    # print("json_text:", json_text)
    # annotated_dict = json.loads(json_text)
    annotated_text_with_reference.append(annotated_text)

In [260]:
# print(annotated_text_with_reference[9])

In [200]:
# Do not remove or delete this file. Overnight's work.
# file_path = 'annotated_text_with_reference_all.json'

# Write JSON data to a file
with open(file_path, 'w') as file:
    json.dump(annotated_text_with_reference, file, indent=4)

print(f"Data successfully written to {file_path}")
# annotated_text_with_reference

Data successfully written to annotated_text_with_reference_all.json


In [261]:
for i in annotated_text_with_reference[20:30]:
    print(i)
    print(

In [213]:
all_paragraphs_report_1 = []

In [13]:
PARAGRAPH_PROMPT = """Break the following text into paragraphs and return the paragraphs: {text}"""
for doc in documents[0:316]:
    prompt_template = ChatPromptTemplate.from_template(PARAGRAPH_PROMPT)
    prompt = prompt_template.format(text=doc.page_content)

    # expanded_query_with_llm - not required in this case
    paragraphs_for_page = model.invoke(prompt)

    # json_prompt = create_prompt(annotated_text, JSON_PARSER_PROMPT_TEMPLATE)
    # json_text = model.invoke(json_prompt)
    # print("json_text:", json_text)
    # annotated_dict = json.loads(json_text)
    all_paragraphs_report_1.append(paragraphs_for_page)

In [264]:
# for i in all_paragraphs:
#     print(i)
#     print("x"*100)

In [211]:
documents[316]

Document(page_content='CEAT LIMITED\nRPG HOUSE463, DR ANNIE BESANT ROAD, WORLI  MUMBAI - 400 030\nWWW.CEAT .COM\nMaking \nMobility Safer & S\nMarter. \nevery d\nay.\nMaking Mobility Safer & S Marter. every day.', metadata={'source': 'ceat/CEAT Limited Annual Report FY16.pdf', 'page': 0})

# Using LlamaParse Markdown output

In [215]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

In [217]:
os.listdir("llama_parse_data")

['2017pdf.md']

In [218]:
markdown_path = "./llama_parse_data/2017pdf.md"
loader = UnstructuredMarkdownLoader(markdown_path)


In [219]:
md_docs = loader.load()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/paruljuniwal/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/paruljuniwal/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [224]:
len(md_docs[0].page_content[0:100])

100

In [263]:
# print(md_docs[0].page_content[0:100000])

In [230]:
all_paragraphs_from_md = []

In [ ]:
PARAGRAPH_PROMPT = """Break the following text into paragraphs and return the paragraphs: {text}"""

md_docs_chunks
for doc in md_docs:
    prompt_template = ChatPromptTemplate.from_template(PARAGRAPH_PROMPT)
    prompt = prompt_template.format(text=doc.page_content)

    # expanded_query_with_llm - not required in this case
    paragraphs_for_page = model.invoke(prompt)

    # json_prompt = create_prompt(annotated_text, JSON_PARSER_PROMPT_TEMPLATE)
    # json_text = model.invoke(json_prompt)
    # print("json_text:", json_text)
    # annotated_dict = json.loads(json_text)
    all_paragraphs_from_md.append(paragraphs_for_page)

In [12]:
# for i in documents:
#     print(len(i.page_content))

In [233]:
documents[0].page_content[0:56]

'TOWARDS A SAFER, \nSMARTER TOMORROW\nAnnual Report 2016-17'

In [234]:
md_docs[0].page_content[0:56]

'CEAT Annual Report 2016-17\n\nTOWARDS A SAFER, SMARTER TOM'

In [235]:
md_docs[0].page_content[0:200]

'CEAT Annual Report 2016-17\n\nTOWARDS A SAFER, SMARTER TOMORROW\n\nCorporate Overview\n\n02 Corporate Review 03 Manufacturing Landscape 04 Expanding Global Footprint 06 Board of Directors 08 Product Portfol'

In [236]:
md = md_docs[0].page_content

In [240]:
md_split = md.split("\n\n")

In [241]:
# print(md[0:10000])

In [242]:
chunk_size = 10000

chunks = [md[i:i+chunk_size] for i in range(0, len(md), chunk_size)]

print(len(chunks))

94


In [243]:
para_from_chunks = []

In [11]:
PARAGRAPH_PROMPT = """Break the following text into paragraphs and return the paragraphs: {text}"""
for chunk in chunks:
    prompt_template = ChatPromptTemplate.from_template(PARAGRAPH_PROMPT)
    prompt = prompt_template.format(text=doc.page_content)

    # expanded_query_with_llm - not required in this case
    paragraphs_for_page = model.invoke(prompt)

    # json_prompt = create_prompt(annotated_text, JSON_PARSER_PROMPT_TEMPLATE)
    # json_text = model.invoke(json_prompt)
    # print("json_text:", json_text)
    # annotated_dict = json.loads(json_text)
    para_from_chunks.append(paragraphs_for_page)

# LLama Parsed Json

In [246]:
file_path = "./llama_parse_data/2016pdf.json"
with open(file_path, 'r') as file:
    json_docs = json.load(file)

In [253]:
pages = json_docs[0]["pages"]

In [265]:
example_text = """Chinese tyre market
China is the world’s largest vehicle market. In 2015, China
recorded a rise of 6% in Passenger Vehicle (PV) sales,
while sales of Commercial Vehicles (CV) dropped 9%.
China accounts for one-third of the global tyre industry
and developments there have huge global implications. In
FY 2015-16, the Chinese tyre market faced tough challenges.
A combination of overcapacities, anti-dumping probes
in major consumer markets such as the U.S., and weaker
domestic demand has made Chinese tyre manufacturers
sell in a distressed manner. This desperation is disrupting
the other markets. China has been the biggest threat to
Indian tyre manufacturers, especially in the TBR (TruckBus Radial) segment. In the absence of anti-dumping duties
being in place, China has been able to supply TBR almost
at the cost of TBB (Truck-Bus Bias) tyres. This has
earned it a healthy market share in India’s TBR replacement
market.
(Source: European Automobile Manufacturers Association)"""

reference_output_with_tags = """Section: Chinese Tyre Market
Title: "Chinese Tyre Market"

Content Summary: Overview of the tyre market dynamics in China, highlighting challenges and the impact on global and Indian markets.

Details:
Market Size: China is described as the world’s largest vehicle market, accounting for one-third of the global tyre industry.
Sales Trends: In 2015, there was a 6% rise in Passenger Vehicle sales, while Commercial Vehicle sales dropped by 9%.
Challenges: The Chinese tyre market faced significant challenges in FY 2015-16, including overcapacities, anti-dumping probes by the U.S., and weaker domestic demand.
Impact: These challenges have led Chinese manufacturers to sell tyres in a distressed manner, disrupting other markets, particularly the Indian TBR (Truck-Bus Radial) market.
Competitive Dynamics: Without anti-dumping duties, China has been able to gain a substantial market share in India’s TBR replacement market by pricing TBR tyres competitively against TBB (Truck-Bus Bias) tyres.
Tags: ["China tyre market", "global impact", "anti-dumping", "market disruption", "Indian tyre market competition"]
"""


In [266]:
PROMPT_TEMPLATE_WITH_EXAMPLE = """
You are a equity research analyst.
You have to structure, annotate and tag the provided annual financial report of a company CEAT which manufactures tyres in India. 
The annotated text with tags with be used to create embeddings for Retrieval Augmented Generation.
Analyse the excerpt and provide a list of comprehensive annotations and tags for the given document.

{excerpt}

---
Use the example input text and format of reference output with tags provided below:
{example_text}
{reference_output}

Provide the list of tags at the end of response as TAGS: [<Tag 1>, <Tag 2>]
"""


In [267]:
def create_prompt(excerpt, example_text, reference_output, prompt_template):
    excerpt_text = f'"""{excerpt}"""'
    prompt_template = ChatPromptTemplate.from_template(prompt_template)
    prompt = prompt_template.format(excerpt=excerpt_text, example_text=example_text, reference_output=reference_output)
    print(prompt)
    print("*" * 100)
    return prompt
    

In [269]:
annotated_text_from_llama = []

In [10]:
model = Ollama(model="llama3", base_url="http://localhost:11434")
for page in pages[0:100]:
    annotation_prompt = create_prompt(page["md"], example_text, reference_output_with_tags, PROMPT_TEMPLATE_WITH_EXAMPLE)
    annotated_text = model.invoke(annotation_prompt)

    # json_prompt = create_prompt(annotated_text, JSON_PARSER_PROMPT_TEMPLATE)
    # json_text = model.invoke(json_prompt)
    # print("json_text:", json_text)
    # annotated_dict = json.loads(json_text)
    annotated_text_from_llama.append(annotated_text)

In [9]:
# for i in annotated_text_from_llama[0:20]:
#     print(i)
#     print("x" * 100)

In [273]:
# print(annotated_text_from_llama[0]

In [342]:
# pages

In [275]:
md_pages = [element["md"] for element in pages]

In [277]:
len(md_pages)

212

In [278]:
len(pages)

212

In [6]:
# md_pages[0]

In [5]:
# md_pages[1]

In [7]:
# md_pages[8]

In [327]:
embedding_function = get_embedding_function()
print(embedding_function)

base_url='http://localhost:11434' model='llama3' embed_instruction='passage: ' query_instruction='query: ' mirostat=None mirostat_eta=None mirostat_tau=None num_ctx=None num_gpu=None num_thread=None repeat_last_n=None repeat_penalty=None temperature=None stop=None tfs_z=None top_k=None top_p=None show_progress=False headers=None model_kwargs=None


In [8]:
# md_pages[0]

In [333]:
db = Chroma.from_documents(md_header_splits, embedding_function, persist_directory=CHROMA_PATH_LLAMA)
# db = Chroma.from_documents(docs, embedding_function)

In [337]:
query = "What is the ebitda OF CEAT?"
docs = db.similarity_search(query)

In [339]:
docs = db.similarity_search_with_score(query)

In [4]:
# docs

In [296]:
CHROMA_PATH_LLAMA = "chroma_llama"
def add_to_chroma_llama_parsed(chunks: list[str]):
    # Load the existing database.
    db = Chroma(
        persist_directory=CHROMA_PATH_LLAMA, embedding_function=get_embedding_function()
    )

    # Calculate Page IDs.
    # chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    # new_chunks = []
    # for chunk in chunks_with_ids:
    #     if chunk.metadata["id"] not in existing_ids:
    #         new_chunks.append(chunk)
    
    if len(chunks):
        print(f"👉 Adding new documents: {len(chunks)}")
        # new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        # print("new_chunk_ids:", new_chunk_ids.split(".pdf:")[-1])
        db.add_documents(chunks)
        db.persist()
    else:
        print("✅ No new documents to add")

In [351]:
# add_to_chroma_llama_parsed(md_pages)

In [292]:
documents[0]

Document(page_content='TOWARDS A SAFER, \nSMARTER TOMORROW\nAnnual Report 2016-17', metadata={'source': 'ceat/CEAT Limited Annual Report FY17.pdf', 'page': 0})

In [ ]:
def calculate_chunk_ids(chunks, source):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for (index, chunk) in enumerate(chunks):
        # source = chunk.metadata.get("source")
        # page = chunk.metadata.get("page")
        current_page_id = f"{source}:{index}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

In [298]:
md_pages[0]

'# CEAT Annual Financial Report\n\n# CEAT Annual Financial Report\n\nCEAT is a leading tyre manufacturing company in India, dedicated to making mobility safer and smarter every day.\n\nOur commitment to innovation and quality has made us a trusted name in the industry. We offer a wide range of tyres for various vehicles, ensuring safety and performance on the road.\n\nAt CEAT, we believe in continuous improvement and sustainable practices. Our financial report reflects our dedication to growth and success in the market.\n\nJoin us in our journey towards safer and smarter mobility. Together, we can make a difference on the roads every day.'

In [312]:
complete_md = "\n\n".join(md_pages)

In [3]:
# complete_md[0:1000]

In [306]:
for (idx, page) in enumerate(md_pages):
    with open(f"./llama_parse_data/md/{idx}.md", "w") as f:
        f.write(md_pages[idx])

In [310]:
# os.listdir()

In [309]:
len(sorted(os.listdir("llama_parse_data/md")))

212

In [304]:
os.mkdir("./llama_parse_data/md")

In [313]:
with open(f"md_pages_from_llama.md", "w") as f:
        f.write(complete_md)

In [314]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [1]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(complete_md)
md_header_splits

In [321]:
md_header_splits[13]

Document(page_content='WIP  \nKey Outsourcing Facilities  \nMap not to scale', metadata={'Header 1': 'CEAT Limited Annual Report 2015-16', 'Header 2': 'Operational Facilities'})

In [324]:
md_header_splits[10]

Document(page_content='CEAT boasts an extensive distribution network to drive growth, with:  \n- 4,300+ Dealers\n- 250+ 2-Wheeler Distributors\n- 400+ Franchisees (CEAT Shoppes and CEAT Hubs)\n- 290+ Multi Brand Outlets & Shop-in-Shop Concepts\n- 600+ Districts covered', metadata={'Header 1': 'CEAT Annual Financial Report', 'Header 2': 'Corporate Overview'})

In [357]:
# for i in md_header_splits:
#     print(i.page_content)
#     print("x"*100)

In [341]:
len(md_header_splits)

435

md_header_splits

In [344]:
md_header_splits[0]

Document(page_content='CEAT is a leading tyre manufacturing company in India, dedicated to making mobility safer and smarter every day.  \nOur commitment to innovation and quality has made us a trusted name in the industry. We offer a wide range of tyres for various vehicles, ensuring safety and performance on the road.  \nAt CEAT, we believe in continuous improvement and sustainable practices. Our financial report reflects our dedication to growth and success in the market.  \nJoin us in our journey towards safer and smarter mobility. Together, we can make a difference on the roads every day.', metadata={'Header 1': 'CEAT Annual Financial Report'})

In [350]:
md_header_splits[5]

Document(page_content='### Forward-looking statements  \nThis report may contain forward-looking statements that set out anticipated results based on the management’s plans and assumptions. We have tried wherever possible to identify such statements by using words such as ‘anticipates’, ‘estimates’, ‘expects’, ‘projects’, ‘intends’, ‘plans’, ‘believes’ and words of similar substance in connection with any discussion of future performance. We cannot guarantee that these forward-looking statements will be realized, although we believe we have been prudent in these assumptions. The achievements of results are subject to risks, uncertainties and', metadata={'Header 1': 'CEAT Annual Financial Report', 'Header 2': 'Financial Statements'})